In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
import re


In [8]:
url = "https://www.google.com/maps"

driver = webdriver.Edge()

driver.get(url)

accept_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "#yDmH0d > c-wiz > div > div > div > div.NIoIEf > div.G4njw > div.AIC7ge > div.CxJub > div.VtwTSb > form:nth-child(2) > div > div > button"))
)

accept_button.click()

search = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "#searchboxinput"))
)
search.send_keys("Restaurants")
search.send_keys(Keys.RETURN)
time.sleep(10)

results =[]
#elems = driver.find_elements(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]')

for i in range(0):
    driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]').send_keys(Keys.PAGE_DOWN)
    time.sleep(1)
elems = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".Nv2PK.THOPZb.CpccDe"))
)
print("Number of feed elements found:", len(elems))
for elem in elems:
    title = elem.find_element(By.CSS_SELECTOR, "div.fontHeadlineSmall")
    results.append(title.text)

links = driver.find_elements(By.CLASS_NAME, "hfpxzc")

links_list = [link.get_attribute("href") for link in links]

driver.close()
print(results)

Number of feed elements found: 9
['Bar en Bistro Raposa', 'Tokyo Sushi', "'t Stoveke", "'t Boerderijtje", "'t Fornuizeke", 'Smul Lounge', 'La Brasseria De Middenstand', 'Lam Thai', 'Carnivor']


In [53]:
adresses = []
ratings = []
reviews_list = []
links_list = ["https://www.google.com/maps/place/Bar+en+Bistro+Raposa/@51.0021478,3.7975561,17z/data=!3m1!4b1!4m6!3m5!1s0x47c375196eb5d1cf:0xdac616be092e319a!8m2!3d51.0021478!4d3.7975561!16s%2Fg%2F11hdmy4jmr?authuser=0&hl=nl&entry=ttu"]
for url in links_list:
    driver = webdriver.Edge()
    driver.get(str(url))
    #Wait for accept cookies button and press accept
    accept_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "#yDmH0d > c-wiz > div > div > div > div.NIoIEf > div.G4njw > div.AIC7ge > div.CxJub > div.VtwTSb > form:nth-child(2) > div > div > button"))
    )
    accept_button.click()
    time.sleep(2)
    #Get address
    adress = driver.find_element(By.CSS_SELECTOR, 'div.Io6YTe.fontBodyMedium.kR99db')
    adresses.append(adress.text)
    #Get rating
    rating = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div/div[1]/div[2]/div/div[1]/div[2]/span[1]/span[1]')
    ratings.append(rating.text)
    #Get reviews
    reviews_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div:nth-child(2) > div > div.e07Vkf.kA9KIf > div > div > div:nth-child(3) > div > div > button:nth-child(2)'))
    )
    reviews_button.click()
    time.sleep(5)
    for i in range(5):
        driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]').send_keys(Keys.PAGE_DOWN)
        time.sleep(1)
    reviews = driver.find_elements(By.CSS_SELECTOR, 'span.wiI7pd')
    print("Number of feed elements found:", len(reviews))
    for elem in reviews:
        reviews_list.append(elem.text)
    driver.quit()



Number of feed elements found: 20


In [58]:
from deep_translator import GoogleTranslator

en_review_list = []
for element in reviews_list:
    translated = GoogleTranslator(source='auto', target='en').translate(text=element)
    en_review_list.append(translated)

In [59]:
print(en_review_list)

['What a wonderful birthday dinner for my husband and myself together with our children and granddaughter💕💕💕. Delicious food😋, delicious wine🍷, super friendly service🙂, cozy atmosphere👌. "FANTASTIC TEAM"💐!!! …', 'Super fun discovery! Sweet service, tasty and tasty food... highly recommended! 👌🏻 …', 'Very tasty food, friendly and cheerful service. You can taste that the food and cocktails are well thought out and the food is really fresh. Very well done!', 'Yesterday I ate a delicious dish with cod and the delicious chocolate mousse afterwards was definitely recommended. The service was smooth and friendly. It was my first introduction to this brasserie and I will definitely return. I will share my positive experience with friends and family. Great! Keep up the good work!', 'Convivial, friendly, well-cared for, delicious and everyone welcome, from young to older. Children are not very welcome in many businesses, but with them a child is a customer who is welcomed with open arms.', 'Alwa

In [69]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
positive = []
neutral = []
negative = []

for element in en_review_list:
    def preprocess(text):
        new_text = []
        for t in text.split(" "):
            t = '@user' if t.startswith('@') and len(t) > 1 else t
            t = 'http' if t.startswith('http') else t
            new_text.append(t)
        return " ".join(new_text)
    MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
    tokenizer = AutoTokenizer.from_pretrained(MODEL)
    config = AutoConfig.from_pretrained(MODEL)
    # PT
    model = AutoModelForSequenceClassification.from_pretrained(MODEL)
    #model.save_pretrained(MODEL)
    text = element
    text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    ranking = np.argsort(scores)
    #ranking = ranking[::-1]
    for i in range(scores.shape[0]):
        l = config.id2label[ranking[i]]
        s = scores[ranking[i]]
        print(f"{i+1}) {l} {np.round(float(s), 4)}")
    positive.append(scores[ranking[2]])
    neutral.append(scores[ranking[1]])
    negative.append(scores[ranking[0]])

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


1) negative 0.0022
2) neutral 0.0054
3) positive 0.9924


In [11]:
import pandas as pd
df = pd.DataFrame(
    {'Name': results,
     'Link': links_list,
     'Adres': adresses

    })

In [14]:
df.head()
df.to_csv("file.csv")